# Capstone Project  - Classifying Authentic and Replica Sneakers: Book 1

Done by: Kenny Lim, GA-DSI15

## Problem Statement

The sneaker resale market is an estimated 2 billion USD in secondary market in 2019. Estimated to be USD6 billion by 2025 according to research firm, Cowen & Co. Due to the lucrative nature of these commodities, there is the inevitable rise of counterfeits. The counterfeit sneakers sales was USD520 billion or 3.3% in global trade in 2018 and it is costing us in various ways. 

The costs to the brands include: 
1. Undercutting sales from the brands. Online counterfeit sales are responsible for approximately USD30.3 billion in annual losses to luxury brands, according to data service firm ResearchAndMarkets’ Global Brand Counterfeiting Report. 
2. Brands have to deal with the backlash from customers who have had bad experiences from fakes. Additional resources have to be allocated to deal with this
3. Reputation damage

Costs to countries:
1. US Customs and Border Protection (CBP) official stated: "However, these items often fund national and transnational criminal organizations, and cost taxpayers billions. " (Oct 2019)

Costs to collectors: 
1. We just want real sneakers man

In light of this, we want to be able to differentiate real and fake sneakers. Our task is to build a classifier that is able to differentiate between real and fake sneakers. Our primary audience will be the sneaker brands and the CBP. 

To do so, we will first be scrapping data from reddit and imgur. We will then ve using a classification model such as CNN to diffentiate between the authentic and the replicas. We will measure our success using several classification metrics including ROCAUC and F1 score. 

With this, we also hope to help buyers inform themselves and to stay away from counterfeits. Empowering the public with information, they will be able to make the right decision which could help to reduce the lucrative nature of fake sneakers. 


## Executive Summary

To combat the increasingly lucrative business of counterfeit sneakers, due to the rise of a sneaker cult following, the goal was to train a Convolutional Neural Network model to classify 2 binary outcome: Authentic or Replica sneakers.

Web scraping was performed to collect a dataset consisting of approximately 20,000 images. The [Data Scraping notebook](./book1_data_scrapping_cleaning.ipynb) can be run periodically to obtain more data over time. Eventually, we came up with a 53/47% split between authentic and replica sneakers respectively. 

As the replicas become more and more accurate to the orignals, the shade of colour and size/shape of details (logos, silhouette) on the sneakers are especially important in our case. The choice was made to maintain the RGB (coloured) and aspect ratios of the images, in the preprocessing of the images 

Our classifier was successful in predicting at an 0.84 ROCAUC score and F1 score of 0.79. We also chose the F1 score because false positive and false negatives were just as damaging to us. 

Overall, there are still areas of improvement but moving forward the first step is definitely to achieve web deployment. 

## Notebooks:
- [Data Scrapping and Cleaning](./book1_data_scrapping_cleaning.ipynb)
- [Image Selection](./book2a_img_selection.ipynb)
- [Image Preprocessing](./book2b_img_preprocessing.ipynb)
- [Modeling and Recommendations](./book3_preprocesing_modeling_recommendations.ipynb)


## Contents:
- [Import Libraries](#Import-Libraries)
- [Data Scrapping](#Data-Scrapping)
- [Data Cleaning](#Data-Cleaning)
- [Image Download](#Image-Download)
- [Save Data to CSV](#Save-Data-to-CSV)

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

#for scrapping imgur and reddit
import requests
import time 
import random

#for image download
import urllib.request
from urllib.error import HTTPError

## Data Scraping

We will be getting data from imgur and reddit. The advantage is that imgur is also categorised by subreddit. Here we have 2 list of subreddits we will be scraping from. This way the data that we obtain can be labeled. 

In [2]:
#list of sub reddits
auth = ["sneakermarket","sneakers","sneakerhead","sneakerscanada",
        "adidas","yeezys","nike","newbalance"]
    
rep = ["repsneakers","sneakerreps", "wengkksneakers",
        "chanzhfsneakers","michaelsneakers"]

#### Get images from Imgur

Lucky for us, imgur has a json API to retrieve links for the images we need. 

In [3]:
%%time

sub_reds = auth + rep                                    #combine list of subreddits


image_url = []                                           #create lists to store scrapped data
subreddit = []

#save links to variables
imgur     = 'http://i.imgur.com/{}{}'
page_api  = 'http://imgur.com/r/{}/new/page/{}/hit.json'
album_api = 'http://imgur.com/ajaxalbums/getimages/{}/hit.json'


for sub_red in sub_reds:                                 #iterate through the list of sub_reds
    s = requests.session()                               #instantiate session
    s.headers['user-agent'] = 'Mozilla/5.0'              #set header so we wont get blocked

    for i in range(5):                                   #iterate through 5 pages for each subreddit
        url = page_api.format(sub_red,i)
        print(f"retrieving url from page {i+1} of {sub_red}...")

        j = s.get(url).json()
        for entry in j['data']:                          #iterate through post in each page
            if entry['ext'] == '.gif' or entry['ext'] == '.mp4':
                pass                                     #if its a gif or video, pass
            else:
                if entry['is_album']:                    #check if its an album
                    url = album_api.format(entry['hash'])
                    j = s.get(url).json()
                    for image in j['data']['images']:    #iterate through album
                        if entry['ext'] == '.gif' or entry['ext'] == '.mp4':
                            pass                         #if its a gif or video, pass
                        else:
                            url = imgur.format(image['hash'], image['ext'])
                            image_url.append(url)        #if not append link to list
                            subreddit.append(sub_red)    #add label
                else:
                    url = imgur.format(entry['hash'], entry['ext'])
                    image_url.append(url)
                    subreddit.append(sub_red)
            

#credit: https://kaijento.github.io/2017/05/08/web-scraping-imgur.com/

retrieving url from page 1 of sneakermarket...
retrieving url from page 2 of sneakermarket...
retrieving url from page 3 of sneakermarket...
retrieving url from page 4 of sneakermarket...
retrieving url from page 5 of sneakermarket...
retrieving url from page 1 of sneakers...
retrieving url from page 2 of sneakers...
retrieving url from page 3 of sneakers...
retrieving url from page 4 of sneakers...
retrieving url from page 5 of sneakers...
retrieving url from page 1 of sneakerhead...
retrieving url from page 2 of sneakerhead...
retrieving url from page 3 of sneakerhead...
retrieving url from page 4 of sneakerhead...
retrieving url from page 5 of sneakerhead...
retrieving url from page 1 of sneakerscanada...
retrieving url from page 2 of sneakerscanada...
retrieving url from page 3 of sneakerscanada...
retrieving url from page 4 of sneakerscanada...
retrieving url from page 5 of sneakerscanada...
retrieving url from page 1 of adidas...
retrieving url from page 2 of adidas...
retrieving

In [4]:
#here we check the number of urls we have scrapped
print(len(image_url))
print(len(subreddit))

12918
12918


#### Get images from Reddit

After scrapping from imgur, i realised that not all users post a photo through imgur and some post directly from reddit itself. So we will be scrapping through reddit as well to increase our authentic sneakers dataset

In [5]:
%%time

#set header so that reddit wont think we are a bot and block us
header = {'User-agent':'Mozilla/5.0'}

#list of subreddits that we want to scrap
sub_reds = auth + rep                                     #change this accordingly to your needs
           

all_posts = []                                            
after = None                                              #set after variable to record last visited page

for sub in sub_reds:
    url = f'https://old.reddit.com/r/{sub}.json'
    posts = []
    pages = 50
    
    print(sub)
    for i in range(pages):                                #due to restriction of 1000 posts per subreddit
        print(f"page {i+1} of {pages}")
        if after == None:
            current_url = url
        else:
            current_url = url + '?after=' + after         #this allows us to go to the next page
        
        res = requests.get(current_url, headers=header)

        if res.status_code != 200:
            print('Status error', res.status_code)
            break

        current_dict = res.json()                         #get json
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)                       #append posts to list
        after = current_dict['data']['after']             #get after

        # generate a random sleep duration to look more 'natural'
        sleep_duration = random.randint(2,10)
        print(f"wait {sleep_duration} seconds...")
        time.sleep(sleep_duration)
        
    all_posts.append(posts)                               #append posts to mainlist

sneakermarket
page 1 of 50
wait 8 seconds...
page 2 of 50
wait 9 seconds...
page 3 of 50
wait 5 seconds...
page 4 of 50
wait 5 seconds...
page 5 of 50
wait 7 seconds...
page 6 of 50
wait 2 seconds...
page 7 of 50
wait 2 seconds...
page 8 of 50
wait 9 seconds...
page 9 of 50
wait 3 seconds...
page 10 of 50
wait 8 seconds...
page 11 of 50
wait 5 seconds...
page 12 of 50
wait 4 seconds...
page 13 of 50
wait 7 seconds...
page 14 of 50
wait 5 seconds...
page 15 of 50
wait 5 seconds...
page 16 of 50
wait 6 seconds...
page 17 of 50
wait 8 seconds...
page 18 of 50
wait 8 seconds...
page 19 of 50
wait 9 seconds...
page 20 of 50
wait 6 seconds...
page 21 of 50
wait 4 seconds...
page 22 of 50
wait 9 seconds...
page 23 of 50
wait 10 seconds...
page 24 of 50
wait 7 seconds...
page 25 of 50
wait 3 seconds...
page 26 of 50
wait 3 seconds...
page 27 of 50
wait 10 seconds...
page 28 of 50
wait 2 seconds...
page 29 of 50
wait 2 seconds...
page 30 of 50
wait 9 seconds...
page 31 of 50
wait 9 seconds...
p

page 6 of 50
wait 9 seconds...
page 7 of 50
wait 7 seconds...
page 8 of 50
wait 2 seconds...
page 9 of 50
wait 2 seconds...
page 10 of 50
wait 3 seconds...
page 11 of 50
wait 6 seconds...
page 12 of 50
wait 5 seconds...
page 13 of 50
wait 4 seconds...
page 14 of 50
wait 2 seconds...
page 15 of 50
wait 2 seconds...
page 16 of 50
wait 3 seconds...
page 17 of 50
wait 8 seconds...
page 18 of 50
wait 10 seconds...
page 19 of 50
wait 8 seconds...
page 20 of 50
wait 9 seconds...
page 21 of 50
wait 6 seconds...
page 22 of 50
wait 2 seconds...
page 23 of 50
wait 9 seconds...
page 24 of 50
wait 2 seconds...
page 25 of 50
wait 5 seconds...
page 26 of 50
wait 9 seconds...
page 27 of 50
wait 4 seconds...
page 28 of 50
wait 10 seconds...
page 29 of 50
wait 6 seconds...
page 30 of 50
wait 6 seconds...
page 31 of 50
wait 8 seconds...
page 32 of 50
wait 4 seconds...
page 33 of 50
wait 3 seconds...
page 34 of 50
wait 2 seconds...
page 35 of 50
wait 7 seconds...
page 36 of 50
wait 10 seconds...
page 37 o

wait 8 seconds...
page 12 of 50
wait 7 seconds...
page 13 of 50
wait 2 seconds...
page 14 of 50
wait 4 seconds...
page 15 of 50
wait 9 seconds...
page 16 of 50
wait 10 seconds...
page 17 of 50
wait 10 seconds...
page 18 of 50
wait 7 seconds...
page 19 of 50
wait 5 seconds...
page 20 of 50
wait 7 seconds...
page 21 of 50
wait 6 seconds...
page 22 of 50
wait 2 seconds...
page 23 of 50
wait 3 seconds...
page 24 of 50
wait 2 seconds...
page 25 of 50
wait 4 seconds...
page 26 of 50
wait 2 seconds...
page 27 of 50
wait 3 seconds...
page 28 of 50
wait 5 seconds...
page 29 of 50
wait 7 seconds...
page 30 of 50
wait 10 seconds...
page 31 of 50
wait 4 seconds...
page 32 of 50
wait 6 seconds...
page 33 of 50
wait 3 seconds...
page 34 of 50
wait 3 seconds...
page 35 of 50
wait 4 seconds...
page 36 of 50
wait 10 seconds...
page 37 of 50
wait 5 seconds...
page 38 of 50
wait 6 seconds...
page 39 of 50
wait 6 seconds...
page 40 of 50
wait 2 seconds...
page 41 of 50
wait 8 seconds...
page 42 of 50
wait

In [6]:
#append imgs and subreddits names for assign labels later

#2 items in list, 1 for each subred
for i in range(len(all_posts)):
    for post in all_posts[i]:
            #append url
            image_url.append(post['url'])

            #subreddit of post
            subreddit.append(post['subreddit'])

In [7]:
#check total number of links and labels
print(len(image_url))
print(len(subreddit))

28248
28248


## Data Cleaning

#### Feature Engineering: Create labels for the dataset

In [8]:
#create col for label
is_rep = ['rep' if sub in rep else 'auth' for sub in subreddit]

In [9]:
#check that the number of labels are correct
len(is_rep)

28248

#### Convert to dataframe

In [10]:
#create new col so that we can drop non images
ext = [x.split('.')[-1] for x in image_url]

#create df
df = pd.DataFrame(list(zip(image_url,subreddit,is_rep,ext)),columns=['url','subreddit','is_rep','ext'])

#drop non img
df = df[(df['ext'] == 'jpg') | (df['ext'] == 'png')]

#convert all labels to lower
df['subreddit'] = df['subreddit'].map(lambda x: x.lower())

#check no of data
df['ext'].value_counts()

jpg    18083
png     1208
Name: ext, dtype: int64

In [11]:
#see number of images for each subreddit
df['subreddit'].value_counts()

sneakerreps        3759
repsneakers        3588
yeezys             2105
sneakers           1431
adidas             1428
sneakermarket      1292
nike               1218
michaelsneakers    1018
newbalance          965
chanzhfsneakers     912
sneakerhead         829
sneakerscanada      448
wengkksneakers      298
Name: subreddit, dtype: int64

In [12]:
#since there maybe some overlap between imgur and reddit,
#we check for duplicates and drop them
df.drop_duplicates('url', inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [13]:
df['subreddit'].value_counts()

sneakerreps        3681
repsneakers        3502
yeezys             1691
adidas             1310
sneakers           1140
nike               1100
sneakermarket      1087
michaelsneakers     922
chanzhfsneakers     831
newbalance          821
sneakerhead         512
sneakerscanada      341
wengkksneakers      203
Name: subreddit, dtype: int64

In [14]:
#let's see our actual number of images
df['is_rep'].value_counts()

rep     9021
auth    8120
Name: is_rep, dtype: int64

In [15]:
#we will use this when we check for existing images
df['name'] = df['url'].map(lambda x: x.split('/')[-1])

## Image Download

In [16]:
#break to prevent running the downloading step accidentally
break

SyntaxError: 'break' outside loop (<ipython-input-16-f8e57580ede3>, line 5)

In [17]:
#import csv to check against existing links 
#this saves us time downloading images that already exists
df_exist = pd.read_csv('../datasets/csv/url_all.csv')
url_exist = list(df_exist['url'].map(lambda x: x.split('/')[-1]))

In [18]:
%%time

# Adding information about user agent
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

#by using a range value, we can tweak the start and end value
#to start/stop if necessary
for i in range(df.shape[0]):
    
    dl_url = df.loc[i,"url"]
    temp_label = df.loc[i,"is_rep"]
    dl_name = df.loc[i,"name"]
    
    if dl_name in url_exist:
        print(f'image {i+1} of {df.shape[0]} exists')
        
    else:
        try:
            #retrieve image and save in images folder
            #change to your own file path accordingly
            urllib.request.urlretrieve(dl_url,f"/Users/kenny/Dropbox/GA/capstone_assets/images/{temp_label}/{dl_name}")
            print(f"downloading {i+1} of {df.shape[0]} images...")

        except HTTPError:
            print(f"failed to download image {i+1} of {df.shape[0]}")
            pass

#credit: https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c

downloading 1 of 17141 images...
downloading 2 of 17141 images...
downloading 3 of 17141 images...
downloading 4 of 17141 images...
downloading 5 of 17141 images...
downloading 6 of 17141 images...
downloading 7 of 17141 images...
downloading 8 of 17141 images...
downloading 9 of 17141 images...
downloading 10 of 17141 images...
downloading 11 of 17141 images...
downloading 12 of 17141 images...
downloading 13 of 17141 images...
downloading 14 of 17141 images...
downloading 15 of 17141 images...
downloading 16 of 17141 images...
downloading 17 of 17141 images...
downloading 18 of 17141 images...
image 19 of 17141 exists
image 20 of 17141 exists
image 21 of 17141 exists
image 22 of 17141 exists
image 23 of 17141 exists
image 24 of 17141 exists
image 25 of 17141 exists
image 26 of 17141 exists
image 27 of 17141 exists
image 28 of 17141 exists
image 29 of 17141 exists
image 30 of 17141 exists
image 31 of 17141 exists
image 32 of 17141 exists
image 33 of 17141 exists
image 34 of 17141 exis

downloading 607 of 17141 images...
downloading 608 of 17141 images...
downloading 609 of 17141 images...
downloading 610 of 17141 images...
downloading 611 of 17141 images...
downloading 612 of 17141 images...
downloading 613 of 17141 images...
downloading 614 of 17141 images...
image 615 of 17141 exists
image 616 of 17141 exists
image 617 of 17141 exists
image 618 of 17141 exists
image 619 of 17141 exists
image 620 of 17141 exists
image 621 of 17141 exists
image 622 of 17141 exists
image 623 of 17141 exists
image 624 of 17141 exists
image 625 of 17141 exists
image 626 of 17141 exists
image 627 of 17141 exists
image 628 of 17141 exists
image 629 of 17141 exists
image 630 of 17141 exists
image 631 of 17141 exists
image 632 of 17141 exists
image 633 of 17141 exists
image 634 of 17141 exists
image 635 of 17141 exists
image 636 of 17141 exists
image 637 of 17141 exists
image 638 of 17141 exists
image 639 of 17141 exists
image 640 of 17141 exists
image 641 of 17141 exists
image 642 of 17141

downloading 1105 of 17141 images...
downloading 1106 of 17141 images...
downloading 1107 of 17141 images...
downloading 1108 of 17141 images...
downloading 1109 of 17141 images...
downloading 1110 of 17141 images...
downloading 1111 of 17141 images...
downloading 1112 of 17141 images...
downloading 1113 of 17141 images...
image 1114 of 17141 exists
image 1115 of 17141 exists
image 1116 of 17141 exists
image 1117 of 17141 exists
image 1118 of 17141 exists
image 1119 of 17141 exists
image 1120 of 17141 exists
image 1121 of 17141 exists
image 1122 of 17141 exists
image 1123 of 17141 exists
image 1124 of 17141 exists
image 1125 of 17141 exists
image 1126 of 17141 exists
image 1127 of 17141 exists
image 1128 of 17141 exists
image 1129 of 17141 exists
image 1130 of 17141 exists
image 1131 of 17141 exists
image 1132 of 17141 exists
image 1133 of 17141 exists
image 1134 of 17141 exists
image 1135 of 17141 exists
image 1136 of 17141 exists
image 1137 of 17141 exists
image 1138 of 17141 exists
i

image 1908 of 17141 exists
image 1909 of 17141 exists
image 1910 of 17141 exists
image 1911 of 17141 exists
image 1912 of 17141 exists
image 1913 of 17141 exists
image 1914 of 17141 exists
image 1915 of 17141 exists
image 1916 of 17141 exists
image 1917 of 17141 exists
image 1918 of 17141 exists
image 1919 of 17141 exists
image 1920 of 17141 exists
image 1921 of 17141 exists
image 1922 of 17141 exists
image 1923 of 17141 exists
image 1924 of 17141 exists
image 1925 of 17141 exists
image 1926 of 17141 exists
image 1927 of 17141 exists
image 1928 of 17141 exists
image 1929 of 17141 exists
image 1930 of 17141 exists
image 1931 of 17141 exists
image 1932 of 17141 exists
image 1933 of 17141 exists
image 1934 of 17141 exists
image 1935 of 17141 exists
image 1936 of 17141 exists
image 1937 of 17141 exists
image 1938 of 17141 exists
image 1939 of 17141 exists
image 1940 of 17141 exists
image 1941 of 17141 exists
image 1942 of 17141 exists
image 1943 of 17141 exists
image 1944 of 17141 exists
i

image 2465 of 17141 exists
image 2466 of 17141 exists
image 2467 of 17141 exists
image 2468 of 17141 exists
image 2469 of 17141 exists
image 2470 of 17141 exists
image 2471 of 17141 exists
image 2472 of 17141 exists
image 2473 of 17141 exists
image 2474 of 17141 exists
image 2475 of 17141 exists
image 2476 of 17141 exists
image 2477 of 17141 exists
image 2478 of 17141 exists
image 2479 of 17141 exists
image 2480 of 17141 exists
image 2481 of 17141 exists
image 2482 of 17141 exists
image 2483 of 17141 exists
image 2484 of 17141 exists
image 2485 of 17141 exists
image 2486 of 17141 exists
image 2487 of 17141 exists
image 2488 of 17141 exists
image 2489 of 17141 exists
image 2490 of 17141 exists
image 2491 of 17141 exists
image 2492 of 17141 exists
image 2493 of 17141 exists
image 2494 of 17141 exists
image 2495 of 17141 exists
image 2496 of 17141 exists
image 2497 of 17141 exists
image 2498 of 17141 exists
image 2499 of 17141 exists
image 2500 of 17141 exists
image 2501 of 17141 exists
i

...

downloading 15937 of 17141 images...
downloading 15938 of 17141 images...
downloading 15939 of 17141 images...
downloading 15940 of 17141 images...
downloading 15941 of 17141 images...
downloading 15942 of 17141 images...
downloading 15943 of 17141 images...
downloading 15944 of 17141 images...
downloading 15945 of 17141 images...
downloading 15946 of 17141 images...
image 15947 of 17141 exists
downloading 15948 of 17141 images...
downloading 15949 of 17141 images...
downloading 15950 of 17141 images...
downloading 15951 of 17141 images...
downloading 15952 of 17141 images...
downloading 15953 of 17141 images...
downloading 15954 of 17141 images...
downloading 15955 of 17141 images...
image 15956 of 17141 exists
downloading 15957 of 17141 images...
downloading 15958 of 17141 images...
downloading 15959 of 17141 images...
downloading 15960 of 17141 images...
downloading 15961 of 17141 images...
downloading 15962 of 17141 images...
downloading 15963 of 17141 images...
downloading 15964 o

downloading 16161 of 17141 images...
downloading 16162 of 17141 images...
downloading 16163 of 17141 images...
downloading 16164 of 17141 images...
downloading 16165 of 17141 images...
downloading 16166 of 17141 images...
downloading 16167 of 17141 images...
downloading 16168 of 17141 images...
downloading 16169 of 17141 images...
downloading 16170 of 17141 images...
downloading 16171 of 17141 images...
downloading 16172 of 17141 images...
downloading 16173 of 17141 images...
downloading 16174 of 17141 images...
downloading 16175 of 17141 images...
downloading 16176 of 17141 images...
downloading 16177 of 17141 images...
downloading 16178 of 17141 images...
downloading 16179 of 17141 images...
downloading 16180 of 17141 images...
downloading 16181 of 17141 images...
downloading 16182 of 17141 images...
downloading 16183 of 17141 images...
downloading 16184 of 17141 images...
downloading 16185 of 17141 images...
downloading 16186 of 17141 images...
downloading 16187 of 17141 images...
d

downloading 16383 of 17141 images...
downloading 16384 of 17141 images...
downloading 16385 of 17141 images...
downloading 16386 of 17141 images...
downloading 16387 of 17141 images...
downloading 16388 of 17141 images...
downloading 16389 of 17141 images...
downloading 16390 of 17141 images...
downloading 16391 of 17141 images...
downloading 16392 of 17141 images...
downloading 16393 of 17141 images...
downloading 16394 of 17141 images...
downloading 16395 of 17141 images...
downloading 16396 of 17141 images...
downloading 16397 of 17141 images...
downloading 16398 of 17141 images...
downloading 16399 of 17141 images...
downloading 16400 of 17141 images...
downloading 16401 of 17141 images...
downloading 16402 of 17141 images...
image 16403 of 17141 exists
downloading 16404 of 17141 images...
downloading 16405 of 17141 images...
downloading 16406 of 17141 images...
downloading 16407 of 17141 images...
downloading 16408 of 17141 images...
downloading 16409 of 17141 images...
downloadin

downloading 16607 of 17141 images...
downloading 16608 of 17141 images...
downloading 16609 of 17141 images...
downloading 16610 of 17141 images...
downloading 16611 of 17141 images...
downloading 16612 of 17141 images...
downloading 16613 of 17141 images...
downloading 16614 of 17141 images...
downloading 16615 of 17141 images...
downloading 16616 of 17141 images...
downloading 16617 of 17141 images...
downloading 16618 of 17141 images...
downloading 16619 of 17141 images...
downloading 16620 of 17141 images...
downloading 16621 of 17141 images...
downloading 16622 of 17141 images...
downloading 16623 of 17141 images...
downloading 16624 of 17141 images...
downloading 16625 of 17141 images...
downloading 16626 of 17141 images...
downloading 16627 of 17141 images...
downloading 16628 of 17141 images...
downloading 16629 of 17141 images...
downloading 16630 of 17141 images...
downloading 16631 of 17141 images...
downloading 16632 of 17141 images...
downloading 16633 of 17141 images...
d

downloading 16830 of 17141 images...
downloading 16831 of 17141 images...
downloading 16832 of 17141 images...
image 16833 of 17141 exists
downloading 16834 of 17141 images...
downloading 16835 of 17141 images...
downloading 16836 of 17141 images...
downloading 16837 of 17141 images...
downloading 16838 of 17141 images...
downloading 16839 of 17141 images...
downloading 16840 of 17141 images...
downloading 16841 of 17141 images...
downloading 16842 of 17141 images...
downloading 16843 of 17141 images...
downloading 16844 of 17141 images...
downloading 16845 of 17141 images...
downloading 16846 of 17141 images...
downloading 16847 of 17141 images...
downloading 16848 of 17141 images...
downloading 16849 of 17141 images...
downloading 16850 of 17141 images...
image 16851 of 17141 exists
downloading 16852 of 17141 images...
image 16853 of 17141 exists
downloading 16854 of 17141 images...
downloading 16855 of 17141 images...
downloading 16856 of 17141 images...
downloading 16857 of 17141 i

## Save to CSV

In [19]:
df = pd.concat([df,df_exist], axis = 0, ignore_index = True)

In [20]:
df.drop_duplicates('url', inplace = True)

In [21]:
df.is_rep.value_counts()

auth    10957
rep      9615
Name: is_rep, dtype: int64

In [22]:
df.shape

(20572, 5)

In [23]:
#update our links so that when we next scrap for data, we wont download images we already have
df.to_csv('../datasets/csv/url_all.csv',index=False)